# Desenvolvimento do modelo

Exploração inicial de modelos de base - CLASSIFICAÇÃO

In [78]:
import pandas as pd
from dagshub.data_engine import datasources
import mlflow
import dagshub
from sklearn.model_selection import train_test_split
import mlflow.sklearn
import mlflow.xgboost
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.svm import SVC
import mlflow.models.signature
from mlflow.models import infer_signature
from sklearn.preprocessing import LabelEncoder
import numpy as np

## Carregando Dataset

In [79]:
ds = datasources.get_datasource("luciancsilva/fiap-10dtsr-mlops-trabalho-final", "processed")

INFO:gql.transport.requests:>>> {"query": "query datasource($id: ID, $name: String) {\n  datasource(id: $id, name: $name) {\n    id\n    name\n    rootUrl\n    integrationStatus\n    preprocessingStatus\n    metadataFields {\n      name\n      valueType\n      multiple\n      tags\n    }\n    type\n  }\n}", "variables": {"id": null, "name": "processed"}}
INFO:gql.transport.requests:<<< {"data":{"datasource":[{"id":6139,"name":"processed","rootUrl":"repo://luciancsilva/fiap-10dtsr-mlops-trabalho-final/main:data/processed","integrationStatus":"VALID","preprocessingStatus":"READY","metadataFields":[{"name":"size","valueType":"INTEGER","multiple":false,"tags":null},{"name":"media type","valueType":"STRING","multiple":false,"tags":null}],"type":"REPOSITORY"}]}}


In [80]:
ds.all().dataframe

INFO:gql.transport.requests:>>> {"query": "query datasource($id: ID, $name: String) {\n  datasource(id: $id, name: $name) {\n    id\n    name\n    rootUrl\n    integrationStatus\n    preprocessingStatus\n    metadataFields {\n      name\n      valueType\n      multiple\n      tags\n    }\n    type\n  }\n}", "variables": {"id": 6139, "name": "processed"}}
INFO:gql.transport.requests:<<< {"data":{"datasource":[{"id":6139,"name":"processed","rootUrl":"repo://luciancsilva/fiap-10dtsr-mlops-trabalho-final/main:data/processed","integrationStatus":"VALID","preprocessingStatus":"READY","metadataFields":[{"name":"size","valueType":"INTEGER","multiple":false,"tags":null},{"name":"media type","valueType":"STRING","multiple":false,"tags":null}],"type":"REPOSITORY"}]}}


Output()

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/luciancsilva/fiap-10dtsr-mlops-trabalho-final "HTTP/1.1 200 OK"
INFO:gql.transport.requests:>>> {"query": "query datasourceQuery($datasource: ID!, $queryInput: QueryInput, $first: Int, $after: String) {\n  datasourceQuery(\n    datasource: $datasource\n    filter: $queryInput\n    first: $first\n    after: $after\n  ) {\n    edges {\n      node {\n        id\n        path\n        metadata {\n          key\n          value\n          timeZone\n        }\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n    queryDataTime\n    selectFields {\n      name\n      originalName\n      autoGenerated\n      valueType\n      asOf\n      multiple\n      tags\n    }\n  }\n}", "variables": {"datasource": 6139, "queryInput": {"query": null}, "first": 5000, "after": null}}


INFO:httpx:HTTP Request: POST https://dagshub.com/api/internal/trackAnalyticsEvent "HTTP/1.1 200 OK"


INFO:gql.transport.requests:<<< {"data":{"datasourceQuery":{"edges":[{"node":{"id":103597804,"path":"credit-score.csv","metadata":[{"key":"media type","value":"text/plain","timeZone":""},{"key":"size","value":27051762,"timeZone":""}]}}],"pageInfo":{"hasNextPage":false,"endCursor":null},"queryDataTime":1754259477,"selectFields":[{"name":"media type","originalName":"media type","autoGenerated":true,"valueType":"STRING","asOf":null,"multiple":false,"tags":null},{"name":"size","originalName":"size","autoGenerated":true,"valueType":"INTEGER","asOf":null,"multiple":false,"tags":null}]}}}


,path,datapoint_id,dagshub_download_url,media type,size
0,credit-score.csv,103597804,https://dagshub.com/api/v1/repos/luciancsilva/...,text/plain,27051762


In [81]:
res = ds.head()

for dp in res:
    dataset_url = dp.download_url

INFO:gql.transport.requests:>>> {"query": "query datasource($id: ID, $name: String) {\n  datasource(id: $id, name: $name) {\n    id\n    name\n    rootUrl\n    integrationStatus\n    preprocessingStatus\n    metadataFields {\n      name\n      valueType\n      multiple\n      tags\n    }\n    type\n  }\n}", "variables": {"id": 6139, "name": "processed"}}
INFO:gql.transport.requests:<<< {"data":{"datasource":[{"id":6139,"name":"processed","rootUrl":"repo://luciancsilva/fiap-10dtsr-mlops-trabalho-final/main:data/processed","integrationStatus":"VALID","preprocessingStatus":"READY","metadataFields":[{"name":"size","valueType":"INTEGER","multiple":false,"tags":null},{"name":"media type","valueType":"STRING","multiple":false,"tags":null}],"type":"REPOSITORY"}]}}


Output()

INFO:gql.transport.requests:>>> {"query": "query datasourceQuery($datasource: ID!, $queryInput: QueryInput, $first: Int, $after: String) {\n  datasourceQuery(\n    datasource: $datasource\n    filter: $queryInput\n    first: $first\n    after: $after\n  ) {\n    edges {\n      node {\n        id\n        path\n        metadata {\n          key\n          value\n          timeZone\n        }\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n    queryDataTime\n    selectFields {\n      name\n      originalName\n      autoGenerated\n      valueType\n      asOf\n      multiple\n      tags\n    }\n  }\n}", "variables": {"datasource": 6139, "queryInput": {"query": null}, "first": 100, "after": null}}


INFO:httpx:HTTP Request: POST https://dagshub.com/api/internal/trackAnalyticsEvent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dagshub.com/api/internal/trackAnalyticsEvent "HTTP/1.1 200 OK"
INFO:gql.transport.requests:<<< {"data":{"datasourceQuery":{"edges":[{"node":{"id":103597804,"path":"credit-score.csv","metadata":[{"key":"media type","value":"text/plain","timeZone":""},{"key":"size","value":27051762,"timeZone":""}]}}],"pageInfo":{"hasNextPage":false,"endCursor":null},"queryDataTime":1754259478,"selectFields":[{"name":"media type","originalName":"media type","autoGenerated":true,"valueType":"STRING","asOf":null,"multiple":false,"tags":null},{"name":"size","originalName":"size","autoGenerated":true,"valueType":"INTEGER","asOf":null,"multiple":false,"tags":null}]}}}


In [82]:
dataset_url

'https://dagshub.com/api/v1/repos/luciancsilva/fiap-10dtsr-mlops-trabalho-final/raw/main/data/processed/credit-score.csv'

In [83]:
df = pd.read_csv(dataset_url)
df.head()

,ID,Customer_ID,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,...,Auto_Loan,Credit-Builder_Loan,Personal_Loan,Home_Equity_Loan,Not_Specified,Mortgage_Loan,Student_Loan,Debt_Consolidation_Loan,Payday_Loan,Missed_Payment_Day
0,0x1602,CUS_0xd40,1,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,...,1,1,1,1,0,0,0,0,0,1
1,0x1603,CUS_0xd40,2,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,...,1,1,1,1,0,0,0,0,0,0
2,0x1604,CUS_0xd40,3,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,...,1,1,1,1,0,0,0,0,0,1
3,0x1605,CUS_0xd40,4,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,...,1,1,1,1,0,0,0,0,0,1
4,0x1606,CUS_0xd40,5,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,...,1,1,1,1,0,0,0,0,0,1


## Verificação e ajuste do Credit_Score para classificação (-1, 0, 1)

In [84]:
# Verificar os valores únicos atuais do Credit_Score
print("Valores únicos no Credit_Score antes do ajuste:")
print(df['Credit_Score'].unique())
print("Tipo atual:", df['Credit_Score'].dtype)

# Se os valores já são -1, 0, 1, apenas garantir que seja int64
# Se são strings como 'Poor', 'Standard', 'Good', mapear para -1, 0, 1
if df['Credit_Score'].dtype == 'object':
    # Mapear strings para valores numéricos (ajuste conforme seus dados)
    credit_score_map = {
        'Poor': -1, 
        'Standard': 0, 
        'Good': 1
    }
    df['Credit_Score'] = df['Credit_Score'].map(credit_score_map)

# Garantir que seja int64
df['Credit_Score'] = df['Credit_Score'].astype('int64')

print("\nValores únicos no Credit_Score após ajuste:")
print(df['Credit_Score'].unique())
print("Distribuição do Credit_Score:")
print(df['Credit_Score'].value_counts().sort_index())

Valores únicos no Credit_Score antes do ajuste:
[ 1  0 -1]
Tipo atual: int64

Valores únicos no Credit_Score após ajuste:
[ 1  0 -1]
Distribuição do Credit_Score:
Credit_Score
-1    28998
 0    53174
 1    17828
Name: count, dtype: int64


## Desenvolvimento e experimentos de modelos

In [85]:
dagshub.init(repo_owner="luciancsilva", repo_name="fiap-10dtsr-mlops-trabalho-final", mlflow=True)

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/luciancsilva/fiap-10dtsr-mlops-trabalho-final "HTTP/1.1 200 OK"


Initialized MLflow to track repo "luciancsilva/fiap-10dtsr-mlops-trabalho-final"

INFO:dagshub:Initialized MLflow to track repo "luciancsilva/fiap-10dtsr-mlops-trabalho-final"


Repository luciancsilva/fiap-10dtsr-mlops-trabalho-final initialized!

INFO:dagshub:Repository luciancsilva/fiap-10dtsr-mlops-trabalho-final initialized!


In [86]:
mlflow.autolog()

2025/08/03 19:17:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/08/03 19:17:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/08/03 19:17:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [87]:
# Usar o LabelEncoder para a coluna 'Payment_of_Min_Amount'
le = LabelEncoder()
df['Payment_of_Min_Amount'] = le.fit_transform(df['Payment_of_Min_Amount'].astype(str))

In [88]:
features = list(df.columns)
features.remove('ID')
features.remove('Customer_ID')
features.remove('Credit_Score')
features.remove('Occupation')
features.remove('Monthly_Inhand_Salary')
features.remove('Interest_Rate')
features.remove('Type_of_Loan')
features.remove('Delay_from_due_date')
features.remove('Changed_Credit_Limit')
features.remove('Num_Credit_Inquiries')
features.remove('Credit_Mix')
features.remove('Amount_invested_monthly')
features.remove('Monthly_Balance')
features.remove('Num_of_Loan')
features.remove('Outstanding_Debt')
features.remove('Not_Specified')
features.remove('Month')

features

['Age',
 'Annual_Income',
 'Num_Bank_Accounts',
 'Num_Credit_Card',
 'Num_of_Delayed_Payment',
 'Credit_Utilization_Ratio',
 'Payment_of_Min_Amount',
 'Total_EMI_per_month',
 'Credit_History_Age_Formated',
 'Auto_Loan',
 'Credit-Builder_Loan',
 'Personal_Loan',
 'Home_Equity_Loan',
 'Mortgage_Loan',
 'Student_Loan',
 'Debt_Consolidation_Loan',
 'Payday_Loan',
 'Missed_Payment_Day']

In [89]:
X = df[features]

In [90]:
len(features)

18

In [91]:
y = df["Credit_Score"]
print("Classes únicas:", y.unique())
print("Tipo do target:", y.dtype)

Classes únicas: [ 1  0 -1]
Tipo do target: int64


In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [93]:
# Mapear classes para XGBoost (que espera classes sequenciais começando do 0)
class_mapping = {-1: 0, 0: 1, 1: 2}
reverse_mapping = {0: -1, 1: 0, 2: 1}

# Aplicar mapeamento nos conjuntos de treino e teste
y_train_xgb = y_train.map(class_mapping)
y_test_xgb = y_test.map(class_mapping)

print("Mapeamento de classes para XGBoost:")
print("Original -> XGBoost")
for orig, xgb in class_mapping.items():
    print(f"{orig} -> {xgb}")
    
print(f"\nDistribuição y_train original: {y_train.value_counts().sort_index()}")
print(f"Distribuição y_train_xgb: {y_train_xgb.value_counts().sort_index()}")

Mapeamento de classes para XGBoost:
Original -> XGBoost
-1 -> 0
0 -> 1
1 -> 2

Distribuição y_train original: Credit_Score
-1    20299
 0    37222
 1    12479
Name: count, dtype: int64
Distribuição y_train_xgb: Credit_Score
0    20299
1    37222
2    12479
Name: count, dtype: int64


In [94]:
def evaluate_and_log_classification_model(kind, model_name, model, X_test, y_test, y_test_original=None, reverse_mapping=None):
    predictions = model.predict(X_test)
    
    # Se for XGBoost, converter predições de volta para classes originais
    if reverse_mapping is not None:
        predictions_original = pd.Series(predictions).map(reverse_mapping)
        y_test_eval = y_test_original
    else:
        predictions_original = predictions
        y_test_eval = y_test
    
    probabilities = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else None

    accuracy = accuracy_score(y_test_eval, predictions_original)
    precision = precision_score(y_test_eval, predictions_original, average='weighted')
    recall = recall_score(y_test_eval, predictions_original, average='weighted')
    f1 = f1_score(y_test_eval, predictions_original, average='weighted')

    mlflow.log_metric("Accuracy", accuracy)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1_Score", f1)

    # Log classification report
    report = classification_report(y_test_eval, predictions_original, output_dict=True)
    for class_label, metrics in report.items():
        if isinstance(metrics, dict):
            for metric_name, value in metrics.items():
                mlflow.log_metric(f"{class_label}_{metric_name}", value)

    signature = infer_signature(X_test, predictions_original)

    if kind == "xgboost":
        mlflow.xgboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "lightgbm":
        mlflow.lightgbm.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    else:
        mlflow.sklearn.log_model(model, model_name, signature=signature, input_example=X_test[:5])

    print(f"Model {model_name} logged with Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    print(f"Predições originais (sample): {predictions_original[:10] if reverse_mapping else predictions[:10]}")

### Experimento com Ridge Classifier

In [95]:
print(X_train.dtypes)

Age                            float64
Annual_Income                  float64
Num_Bank_Accounts              float64
Num_Credit_Card                float64
Num_of_Delayed_Payment         float64
Credit_Utilization_Ratio       float64
Payment_of_Min_Amount            int64
Total_EMI_per_month            float64
Credit_History_Age_Formated    float64
Auto_Loan                        int64
Credit-Builder_Loan              int64
Personal_Loan                    int64
Home_Equity_Loan                 int64
Mortgage_Loan                    int64
Student_Loan                     int64
Debt_Consolidation_Loan          int64
Payday_Loan                      int64
Missed_Payment_Day               int64
dtype: object


In [96]:
with mlflow.start_run(run_name="Ridge Classifier"):
    param_grid = {
        'alpha': [0.1, 1.0, 10.0, 100.0],
        'fit_intercept': [True, False],
        }
    
    ridge = RidgeClassifier()

    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_param("best_alpha", grid_search.best_params_['alpha'])
    mlflow.log_param("best_fit_intercept", grid_search.best_params_['fit_intercept'])

    evaluate_and_log_classification_model("sklearn", "ridge_classifier", best_model, X_test, y_test)

2025/08/03 19:18:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/lucian/Documents/FIAP/fiap-10dtsr-mlops-trabalho-final/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/03 19:18:02 WARNING mlflow.sklearn.utils: Failed to import matplotlib (error: ImportError('generi

Model ridge_classifier logged with Accuracy: 0.5680, Precision: 0.5673, Recall: 0.5680, F1: 0.5169
Predições originais (sample): [0 0 0 0 0 0 0 0 0 1]
🏃 View run Ridge Classifier at: https://dagshub.com/luciancsilva/fiap-10dtsr-mlops-trabalho-final.mlflow/#/experiments/0/runs/818773c585db44a689f75ccf67145055
🧪 View experiment at: https://dagshub.com/luciancsilva/fiap-10dtsr-mlops-trabalho-final.mlflow/#/experiments/0


### Decision Tree Classifier

In [97]:
with mlflow.start_run(run_name="Decision Tree Classifier"):
    param_grid = {
        'max_depth': [None, 3, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    dt = DecisionTreeClassifier(random_state=42)

    grid_search = GridSearchCV(dt, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])

    evaluate_and_log_classification_model("sklearn", "decision_tree_classifier", best_model, X_test, y_test)

2025/08/03 19:19:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/lucian/Documents/FIAP/fiap-10dtsr-mlops-trabalho-final/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/03 19:20:41 WARNING mlflow.sklearn.utils: Failed to import matplotlib (error: ImportError('generi

Model decision_tree_classifier logged with Accuracy: 0.7362, Precision: 0.7362, Recall: 0.7362, F1: 0.7362
Predições originais (sample): [-1  0  1 -1  0 -1  1 -1  0  1]
🏃 View run Decision Tree Classifier at: https://dagshub.com/luciancsilva/fiap-10dtsr-mlops-trabalho-final.mlflow/#/experiments/0/runs/4c0a3d5e4f0948fba1e1d6735a44719f
🧪 View experiment at: https://dagshub.com/luciancsilva/fiap-10dtsr-mlops-trabalho-final.mlflow/#/experiments/0


## XGBoost Classifier (Otimizado)

In [98]:
with mlflow.start_run(run_name="XGBoost_Classifier_Fast"):
   
    # Grid muito mais simples para ser rápido
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 6],
        'learning_rate': [0.1, 0.2]
    }
   
    # Usar dados mapeados para XGBoost
    xgb = XGBClassifier(random_state=42, verbosity=0, eval_metric='mlogloss')
    grid_search = GridSearchCV(xgb, param_grid, scoring='accuracy', cv=3)  # CV reduzido para 3
    grid_search.fit(X_train, y_train_xgb)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_max_depth", best_model.max_depth)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)
    
    # Avaliar com conversão de volta para classes originais
    evaluate_and_log_classification_model("xgboost", "XGBoost Classifier", best_model, X_test, y_test_xgb, y_test, reverse_mapping)

2025/08/03 19:21:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/lucian/Documents/FIAP/fiap-10dtsr-mlops-trabalho-final/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/08/03 19:22:15 WARNING mlflow.sklearn.utils: Failed to import matplotlib (error: ImportError('generi

Model XGBoost Classifier logged with Accuracy: 0.7327, Precision: 0.7323, Recall: 0.7327, F1: 0.7307
Predições originais (sample): 0    0
1    0
2    0
3    0
4    0
5   -1
6    1
7   -1
8    0
9    0
dtype: int64
🏃 View run XGBoost_Classifier_Fast at: https://dagshub.com/luciancsilva/fiap-10dtsr-mlops-trabalho-final.mlflow/#/experiments/0/runs/1f4262978cad410ab7578a265c8a5586
🧪 View experiment at: https://dagshub.com/luciancsilva/fiap-10dtsr-mlops-trabalho-final.mlflow/#/experiments/0


## Registro de Modelo em Produção

In [104]:
# Substitua pelo run_id do melhor modelo
run_id = "4c0a3d5e4f0948fba1e1d6735a44719f"

mlflow.register_model(model_uri=f"runs:/{run_id}/model", name="credit_score_model")

Registered model 'credit_score_model' already exists. Creating a new version of this model...
2025/08/03 19:34:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: credit_score_model, version 2
Created version '2' of model 'credit_score_model'.


<ModelVersion: aliases=[], creation_timestamp=1754260501951, current_stage='None', description='', last_updated_timestamp=1754260501951, name='credit_score_model', run_id='4c0a3d5e4f0948fba1e1d6735a44719f', run_link='', source='mlflow-artifacts:/d89c8d3636e44e82a9744e47bc1ceb8e/4c0a3d5e4f0948fba1e1d6735a44719f/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='2'>